In [0]:
import delta

In [0]:
tabela = dbutils.widgets.get("tabela")
#tabela ="cotacoes"
schema = "dw"
catalog = "bronze"


In [0]:
if not spark.catalog.tableExists(f"{catalog}.{schema}.{tabela}"):

        print("Tabela não existente, criando.")

        df_full = spark.read.format("delta").table(f"raw.db.{tabela}")

        (df_full.coalesce(1)
                .write
                .format("delta")
                .mode("overwrite")
                .option("delta.enableChangeDataFeed", "true")
                .saveAsTable(f"{catalog}.{schema}.{tabela}"))
else:
        print("Tabela já existente, ignorando FULL LOAD")

In [0]:
(spark.read
      .format("delta")
      .option("readChangeFeed", "true")
      .option("startingVersion", 0)  # começa do início
      .table(f"raw.db.{tabela}")
      .createOrReplaceTempView(f"{tabela}_cdc")
      )

query = f'''
SELECT * FROM {tabela}_cdc
WHERE _change_type <> 'update_preimage'
QUALIFY ROW_NUMBER() OVER (PARTITION BY Simbolo ORDER BY _commit_timestamp DESC, _commit_version DESC) = 1;
'''
df_cotacoes_cdc = spark.sql(query)
#df_cotacoes_cdc.display()

In [0]:
bronze = delta.DeltaTable.forName(spark, f"{catalog}.{schema}.{tabela}")

#UPSERT
(bronze.alias("b")
        .merge(df_cotacoes_cdc.alias("c"),"b.Simbolo = c.Simbolo")
        .whenMatchedDelete(condition = "c._change_type = 'delete'")
        .whenMatchedUpdateAll(condition = "c._change_type = 'update_postimage'")
        .whenNotMatchedInsertAll(condition = "c._change_type = 'insert' OR c._change_type = 'update_postimage'")  
        .execute()
)
    